In [ ]:
# # =========================================================
# # FINAL UNIVERSAL MACHINE LEARNING TEMPLATE (TABULAR DATA)
# #
# # Handles:
# # 1) Categorical targets  → Status, Class, NObeyesdad
# # 2) Numerical targets    → price, MEDV
# # 3) Missing values in FEATURES
# # 4) Missing values in TARGET
# # 5) Test WITHOUT target (normal Kaggle)
# # 6) Test WITH target (rare exam case)
# # 7) Single-column output (DEFAULT)
# # 8) Multiclass PROBABILITY output (OPTIONAL – COMMENTED)
# #
# # ❗ IMPORTANT:
# # - This code WORKS AS-IS for normal questions
# # - Multiclass probability block is COMMENTED
# # - Uncomment ONLY if question explicitly asks
# #
# # ❗ EXTRA NOTE (FOR EXAMS):
# # - If test.csv has NO id column
# # - AND question explicitly says "create id starting from 1"
# # - Then generating id using range(1, n+1) is CORRECT
# # =========================================================

# import pandas as pd
# import numpy as np

# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


# # -------------------- STEP 1: Load Data --------------------
# train = pd.read_csv("train.csv")
# test = pd.read_csv("test.csv")


# # -------------------- STEP 2: DEFINE TARGET ----------------
# # 🔴 CHANGE ONLY THIS LINE BASED ON QUESTION
# TARGET = "Status"    # e.g. "Status", "Class", "NObeyesdad", "price"


# # -------------------- STEP 3: CHECK IF TEST CONTAINS TARGET
# # (DO NOT COMMENT)
# test_has_target = TARGET in test.columns
# print("Test contains target:", test_has_target)


# # -------------------- STEP 4: Separate Target & Features ---
# y = train[TARGET]
# train_features = train.drop(columns=[TARGET])

# # Remove rows with missing target (MANDATORY)
# mask = y.notna()
# y = y[mask]
# train_features = train_features.loc[mask]

# # Prepare test features
# if test_has_target:
#     y_test_true = test[TARGET]      # ONLY for evaluation
#     test_features = test.drop(columns=[TARGET])
# else:
#     test_features = test.copy()


# # -------------------- STEP 5: SAVE ORIGINAL TEST IDs -------
# # ❗ Submission id MUST match test.csv id
# #
# # EXAM NOTE:
# # If test.csv DOES NOT have an id column
# # AND question explicitly asks to "create id starting from 1",
# # then range(1, len(test)+1) is CORRECT.
# #
# # KAGGLE NOTE:
# # If test.csv HAS an id column, NEVER generate new ids.
# #
# # if "id" in test.columns:
# #     test_ids = test["id"]
# # else:
# #     test_ids = pd.Series(
# #         range(15000, 15000 + len(test)),
# #         name="id"
# #     )

# if "id" in test.columns:
#     test_ids = test["id"]
# else:
#     test_ids = pd.Series(range(1, len(test) + 1), name="id")


# # -------------------- STEP 6: DROP ID FROM FEATURES --------
# # id is NEVER a feature
# for df in [train_features, test_features]:
#     if "id" in df.columns:
#         df.drop("id", axis=1, inplace=True)


# # -------------------- STEP 7: HANDLE MISSING VALUES --------
# num_cols_train = train_features.select_dtypes(include=np.number).columns
# num_cols_test = test_features.select_dtypes(include=np.number).columns

# train_features[num_cols_train] = train_features[num_cols_train].fillna(
#     train_features[num_cols_train].median()
# )
# test_features[num_cols_test] = test_features[num_cols_test].fillna(
#     test_features[num_cols_test].median()
# )

# cat_cols_train = train_features.select_dtypes(include="object").columns
# cat_cols_test = test_features.select_dtypes(include="object").columns

# for col in cat_cols_train:
#     train_features[col] = train_features[col].fillna(train_features[col].mode()[0])

# for col in cat_cols_test:
#     test_features[col] = test_features[col].fillna(test_features[col].mode()[0])


# # -------------------- STEP 8: ENCODE FEATURES --------------
# for col in cat_cols_train:
#     le = LabelEncoder()
#     train_features[col] = le.fit_transform(train_features[col])

#     test_features[col] = test_features[col].map(
#         lambda x: le.transform([x])[0] if x in le.classes_ else -1
#     )


# # -------------------- STEP 9: TARGET HANDLING --------------
# target_encoder = None
# is_categorical_target = y.dtype == "object"

# if is_categorical_target:
#     target_encoder = LabelEncoder()
#     y = target_encoder.fit_transform(y)
# # Numerical target → nothing to do


# # -------------------- STEP 10: FEATURE SCALING -------------
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(train_features)
# test_scaled = scaler.transform(test_features)


# # -------------------- STEP 11: TRAIN-TEST SPLIT ------------
# X_train, X_val, y_train, y_val = train_test_split(
#     X_scaled, y, test_size=0.2, random_state=42
# )


# # -------------------- STEP 12: MODEL SELECTION -------------
# # RandomForestClassifier → binary + multiclass
# # RandomForestRegressor  → numerical target
# if is_categorical_target:
#     model = RandomForestClassifier(random_state=42)
# else:
#     model = RandomForestRegressor(random_state=42)

# model.fit(X_train, y_train)


# # -------------------- STEP 13: EVALUATION ------------------
# val_pred = model.predict(X_val)

# if is_categorical_target:
#     print("Validation Accuracy:", accuracy_score(y_val, val_pred))
#     print(classification_report(y_val, val_pred))
# else:
#     print("Validation RMSE:", np.sqrt(mean_squared_error(y_val, val_pred)))


# # -------------------- STEP 14: HYPERPARAMETER TUNING -------
# params = {
#     "n_estimators": [100, 200],
#     "max_depth": [None, 10]
# }

# grid = GridSearchCV(model, params, cv=3)
# grid.fit(X_train, y_train)
# best_model = grid.best_estimator_


# # -------------------- STEP 15: PREDICT TEST ----------------
# test_pred = best_model.predict(test_scaled)

# if is_categorical_target and target_encoder is not None:
#     test_pred = target_encoder.inverse_transform(test_pred)


# # =========================================================
# # 🔴 STEP 15B: OPTIONAL TEST EVALUATION
# # (ONLY when test already had target AND question asks)
# # =========================================================
# """
# if test_has_target:
#     print("\n--- Test Set Evaluation ---")
#     if is_categorical_target:
#         print("Test Accuracy:", accuracy_score(y_test_true, test_pred))
#     else:
#         print("Test RMSE:", np.sqrt(mean_squared_error(y_test_true, test_pred)))
# """
# # =========================================================


# # =========================================================
# # 🔴 STEP 16A: SINGLE-COLUMN SUBMISSION (DEFAULT – USE THIS)
# # =========================================================
# submission = pd.DataFrame({
#     "id": test_ids,
#     TARGET: test_pred
# })
# submission.to_csv("submission.csv", index=False)


# # =========================================================
# # 🔴 STEP 16B: MULTICLASS PROBABILITY SUBMISSION (OPTIONAL)
# # ❌ KEEP COMMENTED
# # ❌ UNCOMMENT ONLY IF QUESTION ASKS FOR PROBABILITIES
# # =========================================================
# """
# probs = best_model.predict_proba(test_scaled)

# submission = pd.DataFrame({
#     "id": test_ids,
#     "Status_C":  probs[:, 0],
#     "Status_CL": probs[:, 1],
#     "Status_D":  probs[:, 2]
# })

# submission.to_csv("submission.csv", index=False)
# """
# # =========================================================

# print("submission.csv created successfully")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    accuracy_score, 
    log_loss, 
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.calibration import CalibratedClassifierCV

# ==================== Load Data ====================
df = pd.read_csv("/kaggle/input/mock-test-2-mse-2/train.csv")
df["Status_bin"] = df["Status"].map({"D": 0, "C": 1, "CL": 2})

X = df.drop(["Status", "Status_bin", "id"], axis=1)
y = df["Status_bin"]

# ==================== Encode categorical columns ====================
cat_cols = X.select_dtypes(include=["object"]).columns

# Option 1: Use OrdinalEncoder for all categorical columns at once

oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[cat_cols] = oe.fit_transform(X[cat_cols].astype(str))

# ==================== Fill missing values ====================
X = X.fillna(X.median(numeric_only=True))

# ==================== Cap outliers ====================
def cap_outliers(df, cols, lower=1, upper=99):
    for col in cols:
        low_val = df[col].quantile(lower / 100)
        high_val = df[col].quantile(upper / 100)
        df[col] = df[col].clip(low_val, high_val)
    return df

num_cols = X.select_dtypes(include=['float64', 'int64']).columns
X = cap_outliers(X, num_cols)

# ==================== Train-test split ====================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42, stratify=y
)

# ==================== Random Forest Classifier ====================
num_classes = len(np.unique(y_train))

rf = RandomForestClassifier(
    n_estimators=800,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

# Optional: Use calibrated classifier for probabilities
model = CalibratedClassifierCV(base_estimator=rf, method='isotonic', cv=5)
model.fit(X_train, y_train)

# ==================== Evaluate on train/test ====================
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)


y_pred_prob = model.predict_proba(X_test)
log_l = log_loss(y_test, y_pred_prob, labels=model.classes_)


precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# ======== DISPLAY ========
print("Accuracy:", acc)
print("Log Loss:", log_l)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


# ==================== Prepare test data ====================
df_test = pd.read_csv("/kaggle/input/mock-test-2-mse-2/test.csv")
test_ids = df_test["id"]
x_test = df_test.drop(["id"], axis=1)

# Encode categorical columns same as train
if len(cat_cols) > 0:
    # Use ordinal encoding fitted on train
    x_test[cat_cols] = oe.transform(x_test[cat_cols].astype(str))

# Fill missing values
x_test = x_test.fillna(x_test.median(numeric_only=True))

# ==================== Predict ====================
y_pred1 = model.predict(x_test)
y_pred_prob1 = model.predict_proba(x_test)

# ==================== Create submission ====================
class_labels = model.classes_
submission_cols = [f"Status_{label}" for label in class_labels]
submission = pd.DataFrame(y_pred_prob1, columns=submission_cols)
submission.insert(0, "id", test_ids)
submission = submission.rename(columns={
    "Status_0": "Status_D",
    "Status_1": "Status_C",
    "Status_2": "Status_CL"
})

# Reorder columns: id, Status_C, Status_CL, Status_D
submission = submission[["id", "Status_C", "Status_CL", "Status_D"]]

submission.to_csv("professor_rf.csv", index=False)
print("Submission file saved as professor_rf.csv")
print(submission.head())



# y_pred_test = model.predict(x_test)

# # Convert 0/1/2 back to D / C / CL
# class_map = {0: "D", 1: "C", 2: "CL"}
# y_pred_labels = pd.Series(y_pred_test).map(class_map)

# # ==================== Create final CSV ====================
# submission = pd.DataFrame({
#     "id": test_ids,
#     "Status": y_pred_labels
# })

# submission.to_csv("professor_rf_classes.csv", index=False)
# print("Submission saved as professor_rf_classes.csv")
# print(submission.head())


# # ==================== Predict encoded classes ====================
# y_pred_test = model.predict(x_test)

# # ==================== Create new sequential ID ====================
# new_ids = np.arange(1, len(x_test) + 1)

# # ==================== Create final CSV ====================
# submission = pd.DataFrame({
#     "id": new_ids,
#     "Status": y_pred_test
# })

# submission.to_csv("professor_rf_encoded_classes.csv", index=False)
# print("Saved as professor_rf_encoded_classes.csv")
# print(submission.head())


#  If your dataset is SMALL (<10k rows)
# rf = RandomForestClassifier(
#     n_estimators=300,
#     max_depth=None,
#     min_samples_split=3,
#     min_samples_leaf=1,
#     max_features='sqrt',
#     random_state=42,
#     n_jobs=-1
# )
# ⭐ If your dataset is MEDIUM (10k–100k rows)
# rf = RandomForestClassifier(
#     n_estimators=600,
#     max_depth=20,
#     min_samples_split=5,
#     min_samples_leaf=2,
#     max_features='sqrt',
#     random_state=42,
#     n_jobs=-1
# )
# ⭐ If your dataset is LARGE (>100k rows)
# rf = RandomForestClassifier(
#     n_estimators=1200,
#     max_depth=15,
#     min_samples_split=10,
#     min_samples_leaf=5,
#     max_features='log2',
#     random_state=42,
#     n_jobs=-1
# )



FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/mock-test-2-mse-2/train.csv'